## Ejercicios plotly

In [1]:
import numpy as np
import pandas as pd

import plotly # Para ver la versión
import plotly.express as px

from datetime import datetime

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"plotly=={plotly.__version__}")

**ref**: _https://www.kaggle.com/datasets/matthieugimbert/french-bakery-daily-sales_

In [ ]:
# datasets integrados en plotly:
df = px.data.tips()
df.head()

In [ ]:
df = px.data.iris()
df.head()

In [ ]:
df = px.data.gapminder()
df.head()

### Ejercicio 0:
- Carga el DataFrame **`Bakery sales.csv`**.

In [ ]:
# r'' para evitar problemas con rutas de archivos en windows, donde barras invertidas se pueden interpretar como caracteres de escape
# df = pd.read_csv(r'..\Data\Bakery sales.csv')
df = pd.read_csv('../Data/Bakery sales.csv')
df.head()

### Ejercicio 1:
- Transforma la columna **`unit_price`** a `float`.


- Transforma la columna **`date`** a `datetime`.


- Crea las columnas **`weekday`**, **`month`**, **`year`** y **`hour`**. Las columnas **`weekday`** y **`month`** deben estar representados por nombre, no por número.

In [ ]:
df.info()

In [ ]:
# uso de apply para quitar ' €' y reemplazar , por . y convertir a float
def formatear_precio(price):
    numerical = price.split(' ')[0]
    numerical_point = numerical.replace(',', '.')
    return np.float32(numerical_point)
    
df['unit_price'].apply(formatear_precio)

In [ ]:
df['unit_price'].str.replace(' €', '').str.replace(',','.').astype(np.float32)

In [10]:
df['unit_price'] = df['unit_price'].apply(lambda price: np.float32(price.split(' ')[0].replace(',', '.')))

In [ ]:
# convertir la fecha a datetime
# datetime.strptime('2021-01-02', '%Y-%m-%d')
df['date'].apply(lambda fecha: datetime.strptime(fecha,'%Y-%m-%d'))

In [ ]:
pd.to_datetime(df['date'])

In [13]:
df['date'] = df['date'].astype('datetime64[ns]')

In [ ]:
df.info()

In [ ]:
# year:
df['year'] = df['date'].dt.year
df['year'].head(3)

In [ ]:
# hour (usaremos este para facilitar los gráficos)
df['hour'] = df['time'].apply(lambda time: np.int32(time.split(':')[0]))
df['hour'].head(3)

In [ ]:
# hour decimal
df['time'].apply(lambda time: round(int(time.split(':')[0]) + int(time.split(':')[1]) / 60, 2))

In [18]:
#weekday: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
# df['weekday'] = df['date'].apply(lambda date: date.strftime('%A'))
df['weekday']= df['date'].dt.day_name()

In [ ]:
#month
# df['month'] = df['date'].apply(lambda date: date.strftime('%B'))
df['month']= df['date'].dt.month_name()
df['month'].head()

In [ ]:
df.head(6)

### Ejercicio 2:

¿A que hora se venden más productos?

- Grafica un histograma usando la columna de `hour` y `quantity`.
- Agrega el parámetro `color = year` y `nbins = 48`.
- Haz que el eje X muestre todas las 24 horas del día.

In [ ]:
fig = px.histogram(df, x='hour', y='Quantity', color='year', nbins=48)
fig.update_layout(xaxis= {
    'range': [0, 24],
    'tickvals': list(range(1, 25))
})
## 11 am es cuando más productos se venden

### Ejercicio 3:
- Repite la gráfica anterior, pero quita el filtro por color y separa la gráfica en dos, usando el parámetro `facet_col`.
- Agrega a esa gráfica un violin plot usando el parámetro `marginal`.

In [22]:
# help(px.histogram)

In [ ]:
fig = px.histogram(df, x='hour', y='Quantity', facet_col='year', nbins=48, marginal='rug')
fig.update_layout(xaxis= {
    'range': [0, 24],
    'tickvals': list(range(1, 25))
}, yaxis= {
    'range': [0, 40_000]
})
# fig.update_layout(xaxis_matches='x', yaxis_matches='y')
for axis in fig.layout:
    if axis.startswith('xaxis') or axis.startswith('yaxis'):
        fig.layout[axis].update(range=[0, 24] if axis.startswith('xaxis') else [0, 40000])
        
fig.show()

### Ejercicio 4:

- Filtra el **DataFrame** por las filas donde la columna `article` tenga la palabra **"BAGUETTE"**.
    - ```html
      df[df["article"].str.contains("")]
      ```
    
- Haz una gráfica de **`sunburst`**, separando por año y mes, mostrando la cantidad de elementos vendidos usando el filtro de **"BAGUETTE"**.

In [24]:
df_baguette = df[df['article'].str.contains('BAGUETTE')]

In [ ]:
px.sunburst(df_baguette, values='Quantity', path=['year', 'month'])

In [ ]:
# otro ejemplo:
df_countries = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df_countries, path=['continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df_countries['lifeExp'], weights=df_countries['pop']))
fig.show()

### Ejercicio 5:
- Calcula el precio total de cada fila, multiplicando las columnas `quantity` y `unit_price`. Crea una nueva columna `total_article_price` con esta información.

- Elimina las filas donde `quantity` tiene un outlier.

- Haz un violin plot con esa nueva columna.

In [ ]:
df["total_article_price"] = df["Quantity"] * df["unit_price"]
df.head(4)

In [ ]:
Q1 = df['Quantity'].quantile(0.25)
Q1

In [ ]:
Q3 = df['Quantity'].quantile(0.75)
Q3

In [ ]:
IQR = Q3 - Q1
IQR

In [31]:
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
filtro_outliers = (df['Quantity'] <= upper_limit) & (df['Quantity'] >= lower_limit)
df_no_outliers = df[filtro_outliers]

In [ ]:
# violin plot
px.violin(df, x='total_article_price')

In [ ]:
px.violin(df_no_outliers, x='total_article_price')

### Ejercicio 6:
- Agrupa el **DataFrame** por fecha y obten la suma de la columna **`total_article_price`**. Crea un **DataFrame** nuevo.
- Busca la forma de quedarte con la columna **`weekday`** durante la agrupación.
- Haz un line plot con este **DataFrame**, utiliza también la columna **`weekday`**.

In [ ]:
df.groupby('date')['total_article_price'].sum().plot()

In [ ]:
df_sales = df.groupby('date', as_index=False).agg({
    'total_article_price': 'sum',
    'weekday': 'min'
})
df_sales.head(3)

In [ ]:
px.line(df_sales, x='date', y='total_article_price', color='weekday')

### Ejercicio 7:
- Crea un nuevo DataFrame agrupando por la columna **`ticket_number`**, que tenga como columnas la suma de **`quantity`**, **`total_article_price`** y el maximo de **`year`**. Renombra las columnas.

- Haz un box plot usando la columnas **`quantity`** y **`year`**.

In [ ]:
df_ticket = df.groupby("ticket_number", as_index=False).agg(
    {"Quantity": ["sum"], "total_article_price": ["sum"], "year": ["max"]}
)
df_ticket.head(5)

In [ ]:
#CUIDADO, SOLO PARA CASOS EN LOS QUE SE USA AGG CON [] Y GENERA UN MULTIINDEX EN LOS NOMBRES DE LAS COLUMNAS
df_ticket.columns = [x[0] for x in df_ticket.columns]
df_ticket.head(5)

In [ ]:
# MÁS RÁPIDO Y DIRECTO PORQUE NO HAY QUE RENOMBRAR COLUMNAS, NO USAMOS [] EN LAS AGREGACIONES
df_ticket = df.groupby("ticket_number", as_index=False).agg(
    {"Quantity": "sum", "total_article_price": "sum", "year": "max"}
)
df_ticket.head(5)

In [ ]:
px.box(df_ticket[(df_ticket['Quantity'] >= 0) & (df_ticket['Quantity'] <= 20)], x='year', y='Quantity')

In [ ]:
# USANDO EL DATAFRAME SIN OUTLIERS
px.box(
    df_no_outliers.groupby("ticket_number", as_index=False).agg(
    {"Quantity": "sum", "total_article_price": "sum", "year": "max"}
), x='year', y='Quantity')

In [ ]:
df_ticket.describe()

### Ejercicio 8:
- Haz un pie plot mostrando el total vendido por día de la semana.
- Haz un pie plot mostrando el total vendido por mes.

In [ ]:
df.head()

In [ ]:
df.groupby("weekday", as_index=False).agg({"total_article_price": "sum"}).sort_values('total_article_price',ascending=False)

In [ ]:
# weekday
px.pie(df, values = 'total_article_price', names='weekday')

In [ ]:
# month
px.pie(df, values = 'total_article_price', names='month')

### Ejercicio 9:
- Agrupa los datos por la columna **`article`** y crea un nuevo DataFrame, donde las columnas sean la suma de **`quantity`** y **`total_article_price`** y la media de **`unit_price`** y el mínimo/máximo de **`year`**.
- Encuentra cuales son los 20 elementos más vendidos y muestralos en un bar plot, usa la columna **`total_article_price`** con el parámetro `color`.

In [ ]:
df_article = df.groupby('article', as_index=False).agg({
    'Quantity': 'sum',
    'total_article_price': 'sum',
    'unit_price': 'mean',
    'year': 'min'
})
df_article_top_20 = df_article.sort_values('Quantity', ascending=False).head(20)
df_article_top_20

In [ ]:
px.bar(df_article_top_20, x='article', y='Quantity', color='total_article_price')

In [ ]:
fig = px.bar(
    df_article_top_20, 
    x='article', 
    y='Quantity', 
    color='total_article_price',  
    title='Top 20 Artículos Más Vendidos', 
    labels={'article': 'Artículo', 'Quantity': 'Cantidad Vendida', 'total_article_price_sum': 'Total Precio del Artículo'}
)

fig.show()

### Ejercicio 10:
- Usando el DataFrame, crea una nueva columna **`article_new`**, si el artículo de esa fila no está entre los 20 más vendidos entonces que tenga el nombre de **`OTRO`**, si el artículo está si entre los 20 más vendidos que no se cambie el nombre.

- Repite el bar plot anterior, esta vez usando todas las filas.

In [ ]:
top_20 = df_article_top_20['article'].values
top_20

In [ ]:
# apply sobre df_article['article']
df_article['article_new'] = df_article['article'].apply(lambda article_name: article_name if article_name in top_20 else 'OTRO')
df_article.head(7)

In [ ]:
df_article.head(3)

In [ ]:
df_article_top_20.reset_index(drop=True, inplace=True)
df_article_top_20['article'].index

In [ ]:
df_article_top_20['article'][df_article_top_20['article'] == 'BAGUETTE'].index[0]

In [ ]:
# Opcional: Crear columna indicando la posición en el top 20 
df_article['article_new2'] = df_article['article'].apply(lambda article_name: f'Top {df_article_top_20['article'][df_article_top_20['article'] == article_name].index[0]}' if article_name in top_20 else 'OTRO')
df_article.head(20)

In [ ]:
fig = px.bar(
    df_article.sort_values('Quantity', ascending=False), 
    x='article_new', 
    y='Quantity', 
    color='total_article_price',  
    title='Top 20 Artículos Más Vendidos + Categoría OTROS', 
    labels={'article_new': 'Artículo', 'Quantity': 'Cantidad Vendida', 'total_article_price': 'Total Precio del Artículo'}
)

fig.show()

### Ejercicio 11:
- Comprueba qué relación existe entre la cantidad de productos comprados y el precio total del ticket.
- Aplica una transformación logarítmica en base 10 al precio y dibuja un box plot para cada cantidad de artículos totales.
- Asegúrate de que, si bien el precio sufre una transformación, las anotaciones del plot representen el valor real.

In [ ]:
df.head(4)

In [ ]:
df_ticket = df_no_outliers.groupby("ticket_number", as_index=False).agg(
    {"Quantity": "sum", "total_article_price": "sum"}
)
df_ticket.head(4)

In [ ]:
# Gráfica sin transformación logarítmica
px.box(df_ticket, x='Quantity', y='total_article_price')

In [ ]:
px.histogram(df_ticket, x='total_article_price')

In [ ]:
# Transformación logarítmica:
# si tiene valores 0 o negativos le puede afectar, conviene filtrarlos primero
df_ticket = df_ticket[df_ticket['total_article_price'] > 0]
df_ticket['price_log'] = df_ticket['total_article_price'].apply(np.log)
df_ticket.head(4)

In [ ]:
px.histogram(df_ticket, x='price_log')

In [ ]:
np.exp([-1 , 0, 1, 2, 3, 4, 5])

In [ ]:
fig = px.box(df_ticket, x='Quantity', y='price_log')
fig.update_layout(
    yaxis= {
        'range': [-1 , 5],
        'tickvals': [-1, 0, 1, 2, 3, 4, 5],
        'ticktext': [f'{x:.2f} €' for x in np.exp([-1 , 0, 1, 2, 3, 4, 5])]
    }
)
# si no ponemos np.exp las etiquetas ticktext se mantendrán en la escala logarítmica: -1, 0, 1, 2, 3, 4, 5
#np.exp revertir la transformación logarítmica para regresar a la escala original

In [37]:
################################################################################################################################